In [19]:
import logging
import os
import time
import glob
import shutil
import h5py

import numpy as np
import qtt
import qcodes
import matplotlib.pyplot as plt
from functools import partial    
from  qcodes.plots.qcmatplotlib import MatPlot
from  qcodes.plots.pyqtgraph import QtPlot
from scipy.optimize import curve_fit
import scipy.integrate as integrate
import scipy

import warnings

ModuleNotFoundError: No module named 'qtt'

In [10]:
from qcodes.instrument_drivers.AlazarTech.ATS import AcquisitionController
import math
import numpy as np
import time
# import qtt

In [18]:
#=====================================
# Processing Data for 10MHz IF
#=====================================

# DFT AcquisitionController
class Demodulation_AcquisitionController(AcquisitionController):
    """
    This class represents an example acquisition controller. End users will
    probably want to use something more sophisticated. It will average all
    buffers and then perform a fourier transform on the resulting average trace
    for one frequency component. The amplitude of the result of channel_a will
    be returned.

    Args:
        name: name for this acquisition_conroller as an instrument

        alazar_name: the name of the alazar instrument such that this controller
            can communicate with the Alazar

        demodulation_frequency: the selected component for the fourier transform

        **kwargs: kwargs are forwarded to the Instrument base class

    """
    def __init__(self, name, alazar_name, demodulation_frequency, AWG_name, **kwargs):
        self.demodulation_frequency = demodulation_frequency
        self.acquisitionkwargs = {}
        self.samples_per_record = None
        self.records_per_buffer = None
        self.buffers_per_acquisition = None
        # TODO(damazter) (S) this is not very general:
        self.number_of_channels = 2
        self.cos_list = None
        self.sin_list = None
        self.buffer = None
        # make a call to the parent class and by extension, create the parameter
        # structure of this class
        super().__init__(name, alazar_name, **kwargs)
        self.add_parameter("acquisition", get_cmd=self.do_acquisition)
        self.AWG = self.find_instrument(AWG_name)

    def update_acquisitionkwargs(self, **kwargs):
        """
        This method must be used to update the kwargs used for the acquisition
        with the alazar_driver.acquire
        :param kwargs:
        :return:
        """
        self.acquisitionkwargs.update(**kwargs)

    def do_acquisition(self):
        """
        this method performs an acquisition, which is the get_cmd for the
        acquisiion parameter of this instrument
        :return:
        """
        value = self._get_alazar().acquire(acquisition_controller=self,
                                           **self.acquisitionkwargs)
        return value

    def pre_start_capture(self):
        """
        See AcquisitionController
        :return:
        """
        alazar = self._get_alazar()
        self.samples_per_record = alazar.samples_per_record.get()
        self.records_per_buffer = alazar.records_per_buffer.get()
        self.buffers_per_acquisition = alazar.buffers_per_acquisition.get()
        sample_speed = alazar.get_sample_rate()
        integer_list = np.arange(self.samples_per_record)
        angle_list = (2 * np.pi * self.demodulation_frequency / sample_speed *
                      integer_list)

        self.cos_list = np.cos(angle_list)
        self.sin_list = np.sin(angle_list)
        self.buffer = np.zeros(self.samples_per_record *
                               self.records_per_buffer *
                               self.number_of_channels)

    def pre_acquire(self):
        """
        See AcquisitionController
        :return:
        """
        # this could be used to start an Arbitrary Waveform Generator, etc...
        # using this method ensures that the contents are executed AFTER the
        # Alazar card starts listening for a trigger pulse
        self.AWG.start()

    def handle_buffer(self, data):
        """
        See AcquisitionController
        :return:
        """
        self.buffer += data

    def post_acquire(self):
        """
        See AcquisitionController
        :return:
        """
        alazar = self._get_alazar()
        # average all records over buffers and acquisitions
        records_per_acquisition = (1. * self.buffers_per_acquisition *
                                   self.records_per_buffer)
        recordA = np.zeros(self.samples_per_record)
        for i in range(self.records_per_buffer):
            i0 = i * self.samples_per_record
            i1 = i0 + self.samples_per_record
            recordA += self.buffer[i0:i1] / records_per_acquisition
            # averaged wave of channel A

        recordB = np.zeros(self.samples_per_record)
        for i in range(self.records_per_buffer):
            i0 = i * self.samples_per_record + len(self.buffer) // 2
            i1 = i0 + self.samples_per_record
            recordB += self.buffer[i0:i1] / records_per_acquisition
            # averaged wave of channel B

        if self.number_of_channels == 2:
            '''
            fit channel A and channel B
            S21 = I + 1.j*Q
            '''
            # in case of only 1 mixer
            S21 = self.demodulate_data(recordA, recordB)
            S21m = np.abs(S21)
            S21mag = alazar.signal_to_volt(1,S21m + 127.5)
            # Convert a value from a buffer to an actual value in volts based on the
            # ranges of the channel.
            # +127.5 is just to compensate for how this signal_to_volt function is defined...
            # Takes the input range of the channel into account, should be the same for both channels.
            S21phase = np.angle(S21, deg=True)
            I = alazar.signal_to_volt(1,S21.real + 127.5)
            Q = alazar.signal_to_volt(1,S21.imag + 127.5)
            
            return [S21mag, S21phase, I, Q]
            
#            # in case of 2 mixers
#            res1 = self.fit(recordA) # signal
#            res2 = self.fit(recordB) # reference
#            amplitudeA = alazar.signal_to_volt(1, res1[0] + 127.5)
##            amplitudeB = alazar.signal_to_volt(2, res2[0] + 127.5)
#            phaseA = res1[1]
#            phaseB = res2[1]
#            S21mag = amplitudeA
#            S21phase = (phaseA - phaseB + 180) % 360 - 180 
#            
#            return [S21mag, S21phase, 0, 0]
        else:
            raise Exception("Could not find CHANNEL_B during data extraction")
        return None

    def fit(self, buf):
        """
        Single channel demodulation
        
        :param buf: buffer to perform the transform on
        :return: return amplitude and phase of the resulted transform
        """
        # Useful if considering only on of the 2 quadratures
        RePart = np.dot(buf - 127.5, self.cos_list) / self.samples_per_record
        ImPart = np.dot(buf - 127.5, self.sin_list) / self.samples_per_record

        # the factor of 2 in the amplitude is due to the fact that there is
        # a negative frequency as well
        ampl = 2 * np.sqrt(RePart ** 2 + ImPart ** 2)
        phase = math.atan2(ImPart, RePart) * 360 / (2 * math.pi)

        # see manual page 52!!! (using unsigned data)
        return [ampl, phase]

    def demodulate_data(self, dataA, dataB):
        '''
        Returns a complex point in the IQ plane by integrating and demodulating
        the data.
        
        The -127.5 is to give the right sign to negative inputs.
        The ATS encodes the input in only positive numbers centered around (255-0)/2,
        so recordA and recordB are waves centered around 127.5 (provided AC coupling).
        '''
        I = np.average(self.cos_list * (dataA - 127.5) + self.sin_list * (dataB - 127.5))
        Q = np.average(self.sin_list * (dataA - 127.5) - self.cos_list * (dataB - 127.5))
        return I+1.j*Q
    
    

In [24]:
#alazar = self._get_alazar()
alazar.signal_to_volt(1,32 + 127.5)

NameError: name 'alazar' is not defined